# Vergleich zwischen ZAMG Wetterstationsdaten und KrigR-Daten (ERA5-Land, monatl.)


### <font color=red>**DISCLAIMER**</font>
Die Daten, welche via KrigR in R herunterskaliert wurden, wurden nicht weiter validiert. Hier wird lediglich der "rohe" Output der Daten mit den Wetterstationsdaten von ZAMG DataHub visualisiert!

## Setup:

Um dieses Notebook erfolgreich ausführen zu können, sollten die folgenden Vorbedingungen erfüllt sein:

- Erstellen und Aktivieren eines Conda-Environments: 
    - `conda create -c conda-forge --name ENVIRONMENT-NAME python=3.8 mamba`
    - `conda activate ENVIRONMENT-NAME`<br><br>
    
- Installieren des **geospatial** Packages: 
    - `mamba install -c conda-forge geospatial`

Anschließend kann man mit dem Befehl `jupyter notebook .` den Jupyter Notebook Client lokal öffnen und in einem neuen Notebook den Code ausführen.

## Codeausführung:

### Rastermap:
Als einfachste Lösung muss man an zwei Stellen im Code das gewünschte .tif-File eingeben. In den Kommentaren wurde die Stellen kenntlich gemacht. <br>Zu finden sind die .tif-Files unter dem folgenden Github Repository: https://github.com/AnninaDelacour/bachelor

### Heatmap: 
Die Heatmap braucht keine weiteren Einstellungen. Hier kann man via Datepicker das Datum auswählen :)

<hr>

# Heatmap + KrigR-Rasterdaten 2013

Da die hier verwendete Library `Leafmap` leider nicht die nötige Funktionalität aufweist - und die Handhabung des Codes so simpel wie möglich gehalten werden soll -, ist für den Zeitpunkt des Erstellens des Notebooks die einfachste Lösung, die generierten .tif-Files zum Copy+Paste zur Verfügung zu stellen.<br>
D.h. um einen Vergleich zu ziehen mit der Heatmap (Wetterstationsdatenpunkte, daher sehr stark konzentriert auf einem Standort), muss man manuell in der Raster-Map die .tif-Files austauschen.<br>

In [132]:
import numpy as np
import pandas as pd
import folium
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import leafmap.leafmap as leafmap
import rasterio
from folium.plugins import HeatMap
from IPython.display import display
import branca.colormap as cm

In [145]:
# ------ Start Raster Map ------ #

# Open the GeoTIFF file using rasterio - PUT HERE THE FIRST .tif-FILE FROM GITHUB: https://github.com/AnninaDelacour/bachelor
with rasterio.open('https://github.com/AnninaDelacour/bachelor/blob/main/wspeed_2013/wind_speed_aug_2013.tif?raw=true') as src:
    # Read the data from the GeoTIFF file
    data = src.read()
    # Get the affine transformation matrix for the GeoTIFF
    transform = src.transform
    # Get the CRS for the GeoTIFF
    crs = src.crs

# Create a folium map
r_m = leafmap.Map(location=[47.1133, 11.4147], zoom_start=8.5)

colors = ['#9e0142', '#cb334d', '#f57245', '#fdbf6f', '#fff2aa', '#eaf79e', '#a4daa4', '#54aead', '#4d65ad', '#5e4fa2']
vmin = 0
vmax = 32

r_m.add_colorbar(colors=colors, vmin=vmin, vmax=vmax)

# ADD HERE AGAIN THE (RAW) LINK TO THE .tif-FILE
r_m.add_raster('https://github.com/AnninaDelacour/bachelor/blob/main/wspeed_2013/wind_speed_aug_2013.tif?raw=true', layer_name="Wind", palette="Spectral_r")

# ------ End Raster Map ------ #

In [146]:
# ------ Start Heatmap ------ #

df = pd.read_csv("/Users/annina/Documents/FH/BachelorThesis/R_Files/monatl_wdata_2013.csv")

# Create a new column with the month and year in the format 'YYYY-MM'
df['time'] = pd.to_datetime(df['time'])
df['month_year'] = df['time'].apply(lambda x: x.strftime('%Y-%m'))

# Create a folium map object
m = folium.Map(location=[47.1133, 11.4147], zoom_start=8.5)

colors = ['#9e0142', '#cb334d', '#f57245', '#fdbf6f', '#fff2aa', '#eaf79e', 
              '#a4daa4', '#54aead', '#4d65ad', '#5e4fa2']
vmin = 0.1
vmax = 10.2
    
colormap = cm.LinearColormap(colors=colors, vmin=vmin, vmax=vmax, 
                                 caption='Windgeschwindigkeit (Monatsmittel) in m/s')
    
m.add_child(colormap)
    
svg_style = '<style>svg {background-color: white;}</style>'
m.get_root().header.add_child(folium.Element(svg_style))

# Define a function that will be called when a new month is selected from the dropdown menu
def show_month_heatmap(month):
    # Filter the dataframe to only include data for the selected month
    df_month = df.query("month_year == @month")
    # Create a new heatmap layer using the filtered data
    heatmap = HeatMap(data=df_month[['Latitude', 'Longitude', 'vv']].values.tolist(),
                      name = month,
                      overlay = True,
                      control = True,
                      show = True,
                      radius = 30,
                      blur = 10,
                      gradient = color_dict,
                      )
    heatmap.add_to(m)
    
    
    # Show the map with the new heatmap layer
    display(m)

# ------ End Heatmap ------ #

<hr>

# Raster Map:

## KrigR-Daten - Windgeschwindigkeit in m/s

# How to use the map:

<details>
<summary>Klick hier für die Anleitung:</summary>

![HowTo](https://bachelor.blob.core.windows.net/newcontainer/rastermap.gif "How To:")
</details>

In [147]:
# show the raster map first
display(r_m)

Map(center=[47.14986056251128, 11.483193678511313], controls=(ZoomControl(options=['position', 'zoom_in_text',…

# Heatmap

In [148]:
# show the heatmap second; create the dropdown menu for selecting a month
interact(show_month_heatmap, month=df['month_year'].unique())

interactive(children=(Dropdown(description='month', options=('2013-01', '2013-02', '2013-03', '2013-04', '2013…

<function __main__.show_month_heatmap(month)>